In [3]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
import jieba
import json
from opencc import OpenCC
from gensim.models import word2vec

In [3]:
cc = OpenCC('s2t')
texts = []
for FP in range(4):
    foldName = ''
    if FP == 0:
        foldName = 'AA'
    elif FP == 1:
        foldName = 'AB'
    elif FP == 2:
        foldName = 'AC'
    elif FP == 3:
        foldName = 'AD'

    for i in range(99):
        fileName = ''
        if i < 10:
            fileName = 'D:/JupyterData/wiki_zh/'+ foldName +'/wiki_0'+str(i)
        else:
            fileName = 'D:/JupyterData/wiki_zh/'+ foldName +'/wiki_'+str(i)


        file = open(fileName, 'r', encoding='utf-8')
        dataR = []
        for line in file.readlines():
            dataLine = json.loads(line)
            dataR.append(dataLine)

        for data in dataR:
            texts.append(cc.convert(data['text']))


In [4]:
for ABC in range(0,4):
    foldName = ''
    if FP == 0:
        foldName = 'AI'
    elif FP == 1:
        foldName = 'AJ'
    elif FP == 2:
        foldName = 'AK'
    elif FP == 3:
        foldName = 'AL'        
        
    for i in range(0,99):
        fileName = ''
        if i < 10:
            fileName = 'D:/JupyterData/wiki_zh/'+ foldName +'/wiki_0'+str(i)
        else:
            fileName = 'D:/JupyterData/wiki_zh/'+ foldName +'/wiki_'+str(i)


        file = open(fileName, 'r', encoding='utf-8')
        datas = []
        for line in file.readlines():
            dic = json.loads(line)
            datas.append(dic)

        for data in datas:
            texts.append(cc.convert(data['text']))

In [7]:
select = pd.Series(texts)  
print(select) 

0         數學\n\n數學是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，從某種角度看屬於...
1         哲學\n\n哲學（）是研究普遍的、根本的問題的學科，包括存在、知識、價值、理智、心靈、語言等...
2         文學\n\n文學（），在最廣泛的意義上，是任何單一的書面作品。\n\n更嚴格地說，文學寫作被...
3         歷史\n\n歷史（現代漢語詞彙，古典文言文稱之爲史），指人類社會過去的事件和行動，以及對這些...
4         計算機科學\n\n計算機科學（，有時縮寫爲）是系統性研究信息與計算的理論基礎以及它們在計算機...
5         民族\n\n民族，又稱人，簡稱民、族。\n\n在漢語中，民族一詞具有十分豐富而複雜的內涵，可...
6         戲劇\n\n戲劇是演員將某個故事或情境，以對話、歌唱或動作等方式所表演出來的藝術。戲劇有四個...
7         電影\n\n電影是一種表演藝術、視覺藝術及聽覺藝術，利用膠捲、錄影帶或數位媒體將影像和聲音捕...
8         音樂\n\n音樂，廣義而言，就是指任何以聲音組成的藝術。英文Music一詞源於古希臘語的μο...
9         經濟學\n\n經濟學是一門對產品和服務的生產、分配以及消費進行研究的社會科學。西方語言中的“...
10        政治學\n\n政治學是一門以研究政治行爲、政治體制以及政治相關領域爲主的社會科學學科。在西方...
11        法學\n\n法學（jurisprudence、legal theory）是社會科學中一門特殊...
12        社會學\n\n社會學（）起源於19世紀末期，是一門研究社會的學科。社會學使用各種研究方法進行...
13        軍事學\n\n軍事學與甚多範疇有關，主要與戰爭有關。此外，軍事學本身包含了各種學問。軍事是政...
14        信息學\n\n信息學，舊稱情報學（外來語），主要是指以信息爲研究對象，利用計算機及其程序設計...
15        物理學\n\n物理學（，自然）是研究物質、能量的本質與性質的自然科學。由於物質與能量是所有科...
16        天文學\n\n天文學是一門自然科學，它運用數學、物理和化學等

In [5]:
jiebaTexts = []
with open('seg.txt', 'w', encoding='utf-8') as new_f:
    for text in texts:
        stopwords = {}.fromkeys(['\n','。','(',')','"','、','，','（','）','“','”','《','》',"'",'」','「',';','：','；','…','－','—',',','·'])

        segs = ' '.join(jieba.cut(text, cut_all=False, HMM=True))
        jiebaText = ''
        for seg in segs:
            if seg not in stopwords:
                jiebaText = jiebaText + seg

        jiebaTexts.append(jiebaText)
        new_f.write(jiebaText)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.656 seconds.
Prefix dict has been built successfully.


In [5]:
seed = 728
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 1
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [8]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [9]:
most_similar(model, ['文學'])

,文學,cos
0,詩歌,0.832332
1,散文,0.821422
2,當代,0.807388
3,史學,0.803543
4,現代文學,0.787856
5,新詩,0.778840
6,哲學史,0.777652
7,哲學,0.769549
8,文壇,0.766216
9,著作,0.766063
